Performing DMRG calculations for the H2- and H3 test systems

In [2]:
from pyscf import gto, scf, mcscf
import numpy as np
import matplotlib.pyplot as plt
from pyblock2._pyscf.ao2mo import integrals as itg
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
import os 

Let's start with H2-, first do a single point calc

In [ ]:
H2m = gto.M(atom=[["H", 0, 0, 0],
                  ["H", 1.57, 0, 0]], basis = 'sto-3g', charge = -1, spin = 1)
uhf=scf.UHF(H2m).run()
norbs, nelecas = (2, (2, 1))
ref_ucas = mcscf.UCASSCF(uhf, norbs, nelecas).mc1step()[0]

bond_dims = [10] * 10
noises = [1e-4] *  5 + [1e-5] * 4 + [0]
thrds = [1e-8] * 10
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_uhf_integrals(uhf,
    ncore=0, ncas=None, g2e_symm=1)

#print(h1e)
print("g alpha alpha: ", g2e[0], "and its shape: ", g2e[0].shape)
print("g alpha beta: ", g2e[1], "and its shape: ", g2e[1].shape)
print("g beta beta: ", g2e[2], "and its shape: ", g2e[2].shape)

#dmrg_driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
#dmrg_driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

#mpo = dmrg_driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=1)
#ket = dmrg_driver.get_random_mps(tag="GS", bond_dim=2, nroots=1)
#energy = dmrg_driver.dmrg(mpo, ket, n_sweeps=15, bond_dims=bond_dims, noises=noises,
#   thrds=thrds, cutoff=0, iprint=1)
#print('DMRG energy = %20.15f' % energy)
#print('Delta E(Ucasscf-dmrg): ', ref_ucas-energy)


Plot the energies with respect to the bond dimension

In [ ]:
x1 = np.arange(1, 11, 1)
x2 = np.arange(15, 35, 5)
x = np.concatenate((x1, x2))
Ucasscf_en = [ref_ucas] * 14
dmrg_en = [-0.951163644206081,-0.956369676086010, -0.961565933985533, -0.966237039026747, -0.966237038520761,
           -0.966237032826530, -0.966237035545911, -0.966237036802823, -0.966237028768180, -0.966237038718421,
           -0.966237039069511, -0.966237039104832, -0.966237039046652, -0.966237038248826]

fig, ax = plt.subplots()
ax.plot(x, Ucasscf_en, color='black', label='UHF-UCASSCF(8,3)')
ax.scatter(x, dmrg_en, color='blue', label='UHF-DMRG')
ax.legend(loc='upper right', fontsize=7)

ax.set_xlabel('Bond dimension [-]')
ax.set_ylabel('Energy [Hartree]')
#ax.set_xticks(np.arange(0.5, 4.5, 0.5))
#ax.set_yticks(np.arange(-1.3, -1.7, -0.05))

plt.show()

Let's calculate the dissociation curve

In [ ]:
Euhf_H2 = []
Eucas_H2 = []
Edmrg_H2 = []
Euhfs_H2 = []
Eucass_H2 = []
Edmrgs_H2 = []
mo = None
bond_dims = [10] * 10
noises = [1e-4] *  5 + [1e-5] * 4 + [0]
thrds = [1e-8] * 10

for a in np.arange(0.8, 4.05, 0.05):
    H2minus = gto.M(atom=[["H", 0, 0, 0],
                          [ "H", a, 0, 0]], basis = '6-31g', charge = -1, spin = 1)
    uhf =scf.UHF(H2minus)
    Euhf_H2.append(uhf.kernel())
    norbs, nelecas = (4, (2, 1))
    ucas = mcscf.UCASSCF(uhf, norbs, nelecas)
    if mo is not None:
        mo = mcscf.project_init_guess(ucas, mo)
    Eucas_H2.append(ucas.mc1step(mo)[0])
    
    ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_uhf_integrals(uhf,
    ncore=0, ncas=None, g2e_symm=8)
    dmrg_driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
    dmrg_driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)
    mpo = dmrg_driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=0)
    ket = dmrg_driver.get_random_mps(tag="GS", bond_dim=10, nroots=1)
    energy = dmrg_driver.dmrg(mpo, ket, n_sweeps=15, bond_dims=bond_dims, noises=noises,
    thrds=thrds, cutoff=0, iprint=0)
    Edmrg_H2.append(energy)
    



In [ ]:
x = np.arange(0.8, 4.05, 0.05)

fig, ax = plt.subplots()
ax.scatter(x, Euhf_H2, color='blue', label='UHF H2- 6-31G')
ax.scatter(x, Eucas_H2, color='red', label='UCASSCF(3,8) H2- 6-31G')
ax.scatter(x, Edmrg_H2, color='green', label='UHF-DMRG H2- 6-31G')
ax.legend(loc='upper right', fontsize=8)

ax.set_xlabel('Bondlength [Ångström]')
ax.set_ylabel('Energy [Hartree]')
ax.set_xticks(np.arange(0.8, 4.4, 0.4))
ax.set_yticks(np.arange(-0.9, -0.98, -0.01))

Let's do H3 triangular

In [ ]:
H3tri = gto.M(atom=[["H", 0, 0, 0],
                    ["H", 0.87, 0, 0], 
                    ["H", 0.435, 0.7534421013, 0]], basis = '6-31g', charge = 0, spin = 1)
uhf_h3tri=scf.UHF(H3tri).run()
norbs, nelecas = (6, (2, 1))
ref_ucas = mcscf.UCASSCF(uhf_h3tri, norbs, nelecas).mc1step()[0]

bond_dims = [12] * 10
noises = [1e-4] *  5 + [1e-5] * 5 + [0]
thrds = [1e-8] * 10
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_uhf_integrals(uhf_h3tri,
    ncore=0, ncas=None, g2e_symm=8)

dmrg_driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
dmrg_driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo = dmrg_driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=1)
ket = dmrg_driver.get_random_mps(tag="GS", bond_dim=12, nroots=1)
energy = dmrg_driver.dmrg(mpo, ket, n_sweeps=15, bond_dims=bond_dims, noises=noises,
    thrds=thrds, cutoff=0, iprint=1)
print('DMRG energy = %20.15f' % energy)
print('Delta E(Ucasscf-dmrg): ', ref_ucas-energy)

In [ ]:
Euhf_H3tri = []
Eucas_H3tri = []
Edmrg_H3tri = []
Euhf6_H3tri = []
Eucas6_H3tri = []
Edmrg6_H3tri = []
mo = None
bond_dims = [15] * 10
noises = [1e-4] *  5 + [1e-5] * 4 + [0]
thrds = [1e-8] * 10

for b in np.arange(0.5, 4.05, 0.05):
    H3tri = gto.M(atom=[["H", 0, 0, 0],
                    ["H", 0.87, 0, 0], 
                    ["H", 0.435, b, 0]], basis = '6-31g', charge = 0, spin = 1)
    uhf_h3tri =scf.UHF(H3tri)
    Euhf6_H3tri.append(uhf_h3tri.kernel())
    norbs, nelecas = (6, (2, 1))
    ucas_h3tri = mcscf.UCASSCF(uhf_h3tri, norbs, nelecas)
    if mo is not None:
        mo = mcscf.project_init_guess(ucas_h3tri, mo)
    Eucas6_H3tri.append(ucas_h3tri.mc1step(mo)[0])
    
    ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_uhf_integrals(uhf_h3tri,
    ncore=0, ncas=None, g2e_symm=8)
    dmrg_driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
    dmrg_driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)
    mpo = dmrg_driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=0)
    ket = dmrg_driver.get_random_mps(tag="GS", bond_dim=15, nroots=1)
    energy = dmrg_driver.dmrg(mpo, ket, n_sweeps=15, bond_dims=bond_dims, noises=noises,
    thrds=thrds, cutoff=0, iprint=0)
    Edmrg6_H3tri.append(energy)

In [ ]:
x1 = np.arange(0.5, 4.05, 0.05)

fig, ax = plt.subplots()
ax.scatter(x1, Euhf6_H3tri, color='blue', label='UHF H3 (triangular) 6-31G')
ax.scatter(x1, Eucas6_H3tri, color='red', label='UCASSCF(3,12) H3 (triangular) 6-31G')
ax.scatter(x1, Edmrg6_H3tri, color='green', label='UHF-DMRG H3 (triangular) 6-31G')
ax.legend(loc='upper right', fontsize=8)

ax.set_xlabel('Bondlength [Ångström]')
ax.set_ylabel('Energy [Hartree]')
ax.set_xticks(np.arange(0.5, 4.5, 0.5))
ax.set_yticks(np.arange(-1.425, -1.7, -0.025))

Let's do the same thing again for H3 linear (do testing for the best bond dimension and then do the dissociation curve)

In [ ]:
H3lin = gto.M(atom=[["H", 0, 0, 0],
                    ["H", 0.93, 0, 0], 
                    ["H", -0.93, 0, 0]], basis = '6-31g', charge = 0, spin = 1)
uhf_h3lin=scf.UHF(H3lin).run()
norbs, nelecas = (6, (2, 1))
ref_ucas = mcscf.UCASSCF(uhf_h3lin, norbs, nelecas).mc1step()[0]

bond_dims = [30] * 10
noises = [1e-4] *  5 + [1e-5] * 4 + [0]
thrds = [1e-8] * 10
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_uhf_integrals(uhf_h3lin,
    ncore=0, ncas=None, g2e_symm=8)

dmrg_driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
dmrg_driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo = dmrg_driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=1)
ket = dmrg_driver.get_random_mps(tag="GS", bond_dim=25, nroots=1)
energy = dmrg_driver.dmrg(mpo, ket, n_sweeps=30, bond_dims=bond_dims, noises=noises,
    thrds=thrds, cutoff=0, iprint=1)
print('DMRG energy = %20.15f' % energy)
print('Delta E(Ucasscf-dmrg): ', ref_ucas-energy)

In [ ]:
Euhf_H3lin = []
Eucas_H3lin = []
Edmrg_H3lin = []
Euhf6_H3lin = []
Eucas6_H3lin = []
Edmrg6_H3lin = []

mo = None
norbs, nelecas = (6, (2, 1))
bond_dims = [15] * 10
noises = [1e-4] *  5 + [1e-5] * 4 + [0]
thrds = [1e-8] * 10
for c in np.arange(0.5, 2.05, 0.05):
    H3lin = gto.M(atom=[["H", 0, 0, 0],
                          ["H", c, 0, 0], 
                          ["H", -c, 0, 0]], basis = '6-31g', charge = 0, spin = 1)
    uhf_lin =scf.UHF(H3lin)
    Euhf6_H3lin.append(uhf_lin.kernel())
    ucas_h3lin = mcscf.UCASSCF(uhf_lin, norbs, nelecas)
    if mo is not None:
        mo = mcscf.project_init_guess(ucas_h3lin, mo)
    Eucas6_H3lin.append(ucas_h3lin.mc1step(mo)[0])

    ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_uhf_integrals(uhf_lin,
    ncore=0, ncas=None, g2e_symm=8)
    dmrg_driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
    dmrg_driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)
    mpo = dmrg_driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=0)
    ket = dmrg_driver.get_random_mps(tag="GS", bond_dim=15, nroots=1)
    energy = dmrg_driver.dmrg(mpo, ket, n_sweeps=15, bond_dims=bond_dims, noises=noises,
    thrds=thrds, cutoff=0, iprint=0)
    Edmrg6_H3lin.append(energy)
  

In [ ]:
H3_lin_20 = gto.M(atom=[["H", 0, 0, 0],
                     ["H", 2.0, 0, 0], 
                     ["H", -2.0, 0, 0]], basis = '6-31g', charge = 0, spin = 1)
mf20= scf.UHF(H3_lin_20).run()
dm_init_guess= mf20.make_rdm1()

Euhf_part2 = []
Eucas_part2 = []
Eudmrg_part2 = []
Euhf_631g_part2 = []
Eucas_631g_part2 = []
Edmrg_631g_part2 = []

mo2 = None
norbs2, nelecas = (6, (2, 1))
bond_dims2 = [15] * 10
noises2 = [1e-4] *  5 + [1e-5] * 4 + [0]
thrds2 = [1e-8] * 10
for c in np.arange(2.05, 4.05, 0.05 ):
    H3_lin_2 = gto.M(atom=[["H", 0, 0, 0],
                     ["H", c, 0, 0], 
                     ["H", -c, 0, 0]], basis = '6-31g', charge = 0, spin = 1)
    mf2 = scf.UHF(H3_lin_2)
    Euhf_631g_part2.append(mf2.kernel(dm_init_guess))
    mc2 = mcscf.UCASSCF(mf2, norbs2, nelecas)
    if mo2 is not None:
        mo2 = mcscf.project_init_guess(mc2, mo2)
    Eucas_631g_part2.append(mc2.mc1step(mo)[0])
  
    ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_uhf_integrals(mf2,
    ncore=0, ncas=None, g2e_symm=8)
    dmrg_driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
    dmrg_driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)
    mpo = dmrg_driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=0)
    ket = dmrg_driver.get_random_mps(tag="GS", bond_dim=15, nroots=1)
    energy = dmrg_driver.dmrg(mpo, ket, n_sweeps=15, bond_dims=bond_dims2, noises=noises2,
    thrds=thrds2, cutoff=0, iprint=0)
    Edmrg_631g_part2.append(energy)

In [ ]:
x2 = np.arange(0.5, 2.05, 0.05)
x3 = np.arange(2.05, 4.05, 0.05)

fig, ax = plt.subplots()
ax.scatter(x2, Euhf6_H3lin, color='blue', label='UHF H3 (linear) 6-31G')
ax.scatter(x3, Euhf_631g_part2, color='blue')
ax.scatter(x2, Eucas6_H3lin, color='red', label='UCASSCF(3,12) H3 (linear) 6-31G')
ax.scatter(x3, Eucas_631g_part2, color='red')
ax.scatter(x2, Edmrg6_H3lin, color='green', label='UHF-DMRG H3 (linear) 6-31G')
ax.scatter(x3, Edmrg_631g_part2, color='green')
ax.legend(loc='upper right', fontsize=8)

ax.set_xlabel('Bondlength [Ångström]')
ax.set_ylabel('Energy [Hartree]')
ax.set_xticks(np.arange(0.5, 4.5, 0.5))
ax.set_yticks(np.arange(-1.25, -1.7, -0.05))

DMRG energy test with only the one-body integrals and two-body integrals, first H2- sto.3gm then H3lin sto-3g

In [2]:
alpha_h = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h2_sto3g_min/alpha_htensor.npy')
beta_h = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h2_sto3g_min/beta_htensor.npy')
alpha_transform_h = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h2_sto3g_min/alpha_transformed_htensor.npy')
beta_transform_h = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h2_sto3g_min/beta_transformed_htensor.npy')
alpha_alpha_g = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h2_sto3g_min/alpha_alpha_gtensor.npy')
alpha_beta_g = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h2_sto3g_min/alpha_beta_gtensor.npy')
beta_beta_g = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h2_sto3g_min/beta_beta_gtensor.npy')

print(alpha_beta_g)
#print(beta_h)
#print(alpha_beta_g)

[[[[0.54499896 0.        ]
   [0.         0.55286759]]

  [[0.         0.23400933]
   [0.23400933 0.        ]]]


 [[[0.         0.23400933]
   [0.23400933 0.        ]]

  [[0.55286759 0.        ]
   [0.         0.57537293]]]]


In [5]:
alpha_h_k8 = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h2_sto3g_min_k8/alpha_htensor.npy')
beta_h_k8 = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h2_sto3g_min_k8/beta_htensor.npy')
alpha_alpha_g_k8 = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h2_sto3g_min_k8/alpha_alpha_gtensor.npy')
alpha_beta_g_k8 = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h2_sto3g_min_k8/alpha_beta_gtensor.npy')
beta_beta_g_k8 = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h2_sto3g_min_k8/beta_beta_gtensor.npy')

print(alpha_h_k8)

[[-0.88616835  0.        ]
 [ 0.         -0.66857718]]


In [6]:
alpha_h_k9 = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h2_sto3g_min_k9/alpha_htensor.npy')
beta_h_k9 = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h2_sto3g_min_k9/beta_htensor.npy')
alpha_alpha_g_k9 = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h2_sto3g_min_k9/alpha_alpha_gtensor.npy')
alpha_beta_g_k9 = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h2_sto3g_min_k9/alpha_beta_gtensor.npy')
beta_beta_g_k9 = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h2_sto3g_min_k9/beta_beta_gtensor.npy')

print(alpha_h_k9) 

[[-0.88617079  0.        ]
 [ 0.         -0.66857765]]


In [4]:
#h1_npyfile = (alpha_h, beta_h)
#h1_k8_npy = (alpha_h_k8, beta_h_k8)
#h1_transformed_npyfile = (alpha_transform_h, beta_transform_h)
#g2_npyfile = (alpha_alpha_g, alpha_beta_g, beta_beta_g)
#g2_k8_npy = (alpha_alpha_g_k8, alpha_beta_g_k8, beta_beta_g_k8)
#h1_k9_npy = (alpha_h_k9, beta_h_k9)
#g2_k9_npy = (alpha_alpha_g_k9, alpha_beta_g_k9, beta_beta_g_k9)

H2m = gto.M(atom=[["H", 0, 0, 0],
                  ["H", 1.57, 0, 0]], basis = 'sto-3g', charge = -1, spin = 1)
uhf=scf.UHF(H2m).run()
norbs, nelecas = (2, (2, 1))
ref_ucas = mcscf.UCASSCF(uhf, norbs, nelecas).mc1step()[0]

bond_dims = [10] * 10
noises = [1e-4] *  5 + [1e-5] * 5 + [0]
thrds = [1e-8] * 10
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_uhf_integrals(uhf,
    ncore=0, ncas=None, g2e_symm=1)


dmrg_driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
dmrg_driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo = dmrg_driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=0)
ket = dmrg_driver.get_random_mps(tag="GS", bond_dim=10, nroots=1)
energy = dmrg_driver.dmrg(mpo, ket, n_sweeps=15, bond_dims=bond_dims, noises=noises,
   thrds=thrds, cutoff=0, iprint=0)
print('UHF-DMRG energy = %20.15f' % energy)
#print(dmrg_driver.get_1pdm(ket))
#print(dmrg_driver.get_2pdm(ket))


"""
dmrg_driver_test = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
dmrg_driver_test.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo_test = dmrg_driver_test.get_qc_mpo(h1e=h1_npyfile, g2e=g2_npyfile, ecore=ecore, iprint=0)
ket_test = dmrg_driver_test.get_random_mps(tag="GS", bond_dim=10, nroots=1)
energy_test = dmrg_driver_test.dmrg(mpo_test, ket_test, n_sweeps=15, bond_dims=bond_dims, noises=noises,
   thrds=thrds, cutoff=0, iprint=0)
print('UHF-MRA-DMRG energy (k7) = %20.15f' % energy_test)

dmrg_driver_k8 = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
dmrg_driver_k8.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo_k8 = dmrg_driver_k8.get_qc_mpo(h1e=h1_k8_npy, g2e=g2_k8_npy, ecore=ecore, iprint=0)
ket_k8 = dmrg_driver_k8.get_random_mps(tag="GS", bond_dim=10, nroots=1)
energy_k8 = dmrg_driver_k8.dmrg(mpo_k8, ket_k8, n_sweeps=15, bond_dims=bond_dims, noises=noises,
   thrds=thrds, cutoff=0, iprint=0)
print('UHF-MRA-DMRG energy (k8) = %20.15f' % energy_k8)

dmrg_driver_k9 = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
dmrg_driver_k9.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo_k9 = dmrg_driver_k9.get_qc_mpo(h1e=h1_k9_npy, g2e=g2_k9_npy, ecore=ecore, iprint=0)
ket_k9 = dmrg_driver_k9.get_random_mps(tag="GS", bond_dim=10, nroots=1)
energy_k9 = dmrg_driver_k9.dmrg(mpo_k8, ket_k9, n_sweeps=15, bond_dims=bond_dims, noises=noises,
   thrds=thrds, cutoff=0, iprint=0)
print('UHF-MRA-DMRG energy (k9) = %20.15f' % energy_k9) """


#print('Delta E-k7: ', energy-energy_test)
#print('Delta E-k8: ', energy-energy_k8)
#print('Delta E-k9: ', energy-energy_k9)
#print(h1e[0])

converged SCF energy = -0.687169754954451  <S^2> = 0.75  2S+1 = 2
UCASSCF energy = -0.687169754954451
UCASCI E = -0.687169754954451  E(CI) = -1.02422530331114  S^2 = 0.7500000
UHF-DMRG energy =   -0.687169754954451


'\ndmrg_driver_test = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)\ndmrg_driver_test.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)\n\nmpo_test = dmrg_driver_test.get_qc_mpo(h1e=h1_npyfile, g2e=g2_npyfile, ecore=ecore, iprint=0)\nket_test = dmrg_driver_test.get_random_mps(tag="GS", bond_dim=10, nroots=1)\nenergy_test = dmrg_driver_test.dmrg(mpo_test, ket_test, n_sweeps=15, bond_dims=bond_dims, noises=noises,\n   thrds=thrds, cutoff=0, iprint=0)\nprint(\'UHF-MRA-DMRG energy (k7) = %20.15f\' % energy_test)\n\ndmrg_driver_k8 = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)\ndmrg_driver_k8.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)\n\nmpo_k8 = dmrg_driver_k8.get_qc_mpo(h1e=h1_k8_npy, g2e=g2_k8_npy, ecore=ecore, iprint=0)\nket_k8 = dmrg_driver_k8.get_random_mps(tag="GS", bond_dim=10, nroots=1)\nenergy_k8 = dmrg_driver_k8.dmrg(mpo_k8, ket_k8, n_sweeps=15, bond_dims=bond_dims, noises=nois

In [6]:
#use this carefully to delete the scratch files

folder_path = "./tmp"

for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Check if it's a file (not a directory) and delete it
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"Deleted: {file_path}")



Deleted: ./tmp/F.MPS.GS.0
Deleted: ./tmp/F.MPS.GS.1
Deleted: ./tmp/F.MPS.GS.2
Deleted: ./tmp/F.MPS.INFO.GS.LEFT.0
Deleted: ./tmp/F.MPS.INFO.GS.RIGHT.0
Deleted: ./tmp/F.MPS.INFO.GS.LEFT.1
Deleted: ./tmp/F.MPS.INFO.GS.RIGHT.1
Deleted: ./tmp/F.MPS.INFO.GS.LEFT.2
Deleted: ./tmp/F.MPS.INFO.GS.RIGHT.2
Deleted: ./tmp/F.MPS.INFO.GS.LEFT.3
Deleted: ./tmp/F.MPS.INFO.GS.RIGHT.3
Deleted: ./tmp/F.MPS.GS.-1
Deleted: ./tmp/GS-mps_info.bin
Deleted: ./tmp/F.MPS.INFO.PDM-KET@TMP.LEFT.0
Deleted: ./tmp/F.MPS.INFO.PDM-KET@TMP.RIGHT.0
Deleted: ./tmp/F.MPS.INFO.PDM-KET@TMP.LEFT.1
Deleted: ./tmp/F.MPS.INFO.PDM-KET@TMP.RIGHT.1
Deleted: ./tmp/F.MPS.INFO.PDM-KET@TMP.LEFT.2
Deleted: ./tmp/F.MPS.INFO.PDM-KET@TMP.RIGHT.2
Deleted: ./tmp/F.MPS.INFO.PDM-KET@TMP.LEFT.3
Deleted: ./tmp/F.MPS.INFO.PDM-KET@TMP.RIGHT.3
Deleted: ./tmp/F.MPS.PDM-KET@TMP.0
Deleted: ./tmp/F.MPS.PDM-KET@TMP.1
Deleted: ./tmp/F.MPS.PDM-KET@TMP.2
Deleted: ./tmp/F.MPS.PDM-KET@TMP.-1
Deleted: ./tmp/F0.PART.NPDM.NPDM.FRAG.1.fpc
Deleted: ./tmp/F0.PART.

In [17]:
#export 1- and 2-rdms

onerdm_pyscf = dmrg_driver.get_1pdm(ket)
twordm_pyscf = dmrg_driver.get_2pdm(ket)

alpha_1rdm=onerdm_pyscf[0]
beta_1rdm=onerdm_pyscf[1]
aa_2rdm = twordm_pyscf[0]
ab_2rdm = twordm_pyscf[1]
bb_2rdm = twordm_pyscf[2]
alpha_alpha_2rdm = twordm_pyscf[0].transpose(0, 3, 1, 2)
alpha_alpha_2rdm = np.swapaxes(alpha_alpha_2rdm, 1, 2)
alpha_beta_2rdm = twordm_pyscf[1].transpose(0, 3, 1, 2)
alpha_beta_2rdm = np.swapaxes(alpha_beta_2rdm, 1, 2)
beta_beta_2rdm = twordm_pyscf[2].transpose(0, 3, 1, 2)
beta_beta_2rdm = np.swapaxes(beta_beta_2rdm, 1, 2)
aa_g2 = dmrg_driver.unpack_g2e(alpha_alpha_g)
ab_g2 = dmrg_driver.unpack_g2e(alpha_beta_g)
bb_g2 = dmrg_driver.unpack_g2e(beta_beta_g)
swapped_aag = np.swapaxes(alpha_alpha_g, 1,2)


energy_from_int = np.einsum('ij,ij->', alpha_1rdm, alpha_h) + np.einsum('ij,ij->', beta_1rdm, beta_h) + 0.5 * (np.einsum('ijkl,ikjl->', alpha_alpha_2rdm, aa_g2) + 2 *
                                                                                         np.einsum('ijkl,ikjl->', alpha_beta_2rdm, ab_g2) + 
                                                                                         np.einsum('ijkl,ikjl->', beta_beta_2rdm, bb_g2)) + 3.370573e-01
energy_testing = np.einsum('ij,ij->', alpha_1rdm, alpha_h) + np.einsum('ij,ij->', beta_1rdm, beta_h) + 0.5 * (np.einsum('ijkl,ikjl->', aa_2rdm, aa_g2) + 2 *
                                                                                         np.einsum('ijkl,ikjl->', ab_2rdm, ab_g2) + 
                                                                                         np.einsum('ijkl,ikjl->', bb_2rdm, bb_g2)) + 3.370573e-01
energy_from_pyscf = np.einsum('ij,ij->', alpha_1rdm, h1e[0]) + np.einsum('ij,ij->', beta_1rdm, h1e[1]) + 0.5 * (np.einsum('ijkl,ijkl->', alpha_alpha_2rdm, dmrg_driver.unpack_g2e(g2e[0])) + 
                                                                                         np.einsum('ijkl,ijkl->', alpha_beta_2rdm, dmrg_driver.unpack_g2e(g2e[1])) + 
                                                                                         np.einsum('ijkl,ijkl->', beta_beta_2rdm, dmrg_driver.unpack_g2e(g2e[2]))) + 3.370573e-01
two_electron_energy = 0.5 * (np.einsum('ijkl,ikjl->', alpha_alpha_2rdm, aa_g2) + 2 * np.einsum('ijkl,ikjl->', alpha_beta_2rdm, ab_g2) + np.einsum('ijkl,ikjl->', beta_beta_2rdm, bb_g2))
two_el_unshuffled = 0.5 * (np.einsum('ijkl,ikjl->', aa_2rdm, aa_g2) + 2 * np.einsum('ijkl,ikjl->', ab_2rdm, ab_g2) + np.einsum('ijkl,ikjl->', bb_2rdm, bb_g2))
aa_contr = 0.5 * (np.einsum('ijkl,ikjl->', alpha_alpha_2rdm, aa_g2))
aa_contr_test = 0.5 * (np.einsum('ijkl,ijkl->', alpha_alpha_2rdm, swapped_aag))

#print(energy_from_int)
#print(energy_testing)
#print("Two electron energy: ", two_electron_energy)
#print("Two-el unshuffled: ", two_el_unshuffled)
#print(aa_contr)
#print(aa_contr_test)
#print(swapped_aag)
print(ab_2rdm )
print(alpha_beta_2rdm)
#print(energy_from_pyscf)

#np.save('/workspaces/MRA-OrbitalOptimization/reduced_density_matrices/h2_sto3g_min/alpha_rdms.npy', alpha_1rdm)
#np.save('/workspaces/MRA-OrbitalOptimization/reduced_density_matrices/h2_sto3g_min/beta_rdms.npy', beta_1rdm)
#np.save('/workspaces/MRA-OrbitalOptimization/reduced_density_matrices/h2_sto3g_min/alpha_alpha_rdms.npy', alpha_alpha_2rdm)
#np.save('/workspaces/MRA-OrbitalOptimization/reduced_density_matrices/h2_sto3g_min/alpha_beta_rdms.npy', alpha_beta_2rdm)
#np.save('/workspaces/MRA-OrbitalOptimization/reduced_density_matrices/h2_sto3g_min/beta_beta_rdms.npy', beta_beta_2rdm)

[[[[1. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 1.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]]
[[[[1. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]]]


 [[[0. 0.]
   [1. 0.]]

  [[0. 0.]
   [0. 0.]]]]


In [ ]:
print(alpha_alpha_2rdm.shape)


(2, 2, 2, 2)


In [18]:
alpha_h_h3lin = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h3lin_sto3g_min_k7/alpha_htensor.npy')
beta_h_h3lin = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h3lin_sto3g_min_k7/beta_htensor.npy')
alpha_alpha_g_h3lin = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h3lin_sto3g_min_k7/alpha_alpha_gtensor.npy')
alpha_beta_g_h3lin = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h3lin_sto3g_min_k7/alpha_beta_gtensor.npy')
beta_beta_g_h3lin = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h3lin_sto3g_min_k7/beta_beta_gtensor.npy')

print(alpha_h_h3lin)
#print(beta_h_h3lin.shape)
#print(alpha_alpha_g_h3lin)
#print(alpha_beta_g_h3lin)
#print(beta_beta_g_h3lin.shape)


[[-1.56300332  0.          0.        ]
 [ 0.         -1.18409705  0.        ]
 [ 0.          0.         -0.74500386]]


In [4]:
alpha_h3lin_k8 = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h3lin_sto3g_min_k8/alpha_htensor.npy')
beta_h3lin_k8 = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h3lin_sto3g_min_k8/beta_htensor.npy')
alpha_alpha_h3lin_k8 = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h3lin_sto3g_min_k8/alpha_alpha_gtensor.npy')
alpha_beta_h3lin_k8 = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h3lin_sto3g_min_k8/alpha_beta_gtensor.npy')
beta_beta_h3lin_k8 = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h3lin_sto3g_min_k8/beta_beta_gtensor.npy')

print(alpha_h3lin_k8)

[[-1.56300644  0.          0.        ]
 [ 0.         -1.18409472  0.        ]
 [ 0.          0.         -0.74500026]]


In [5]:
alpha_h3lin_k9 = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h3lin_sto3g_min_k9/alpha_htensor.npy')
beta_h3lin_k9 = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h3lin_sto3g_min_k9/beta_htensor.npy')
alpha_alpha_h3lin_k9 = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h3lin_sto3g_min_k9/alpha_alpha_gtensor.npy')
alpha_beta_h3lin_k9 = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h3lin_sto3g_min_k9/alpha_beta_gtensor.npy')
beta_beta_h3lin_k9 = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h3lin_sto3g_min_k9/beta_beta_gtensor.npy')

In [10]:
alpha_h3lin_k7_test = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h3lin_sto3g_k7_test/alpha_htensor.npy')
beta_h3lin_k7_test = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h3lin_sto3g_k7_test/beta_htensor.npy')
alpha_alpha_h3lin_k7_test = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h3lin_sto3g_k7_test/alpha_alpha_gtensor.npy')
alpha_beta_h3lin_k7_test = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h3lin_sto3g_k7_test/alpha_beta_gtensor.npy')
beta_beta_h3lin_k7_test = np.load('/workspaces/MRA-OrbitalOptimization/output_folder/h3lin_sto3g_k7_test/beta_beta_gtensor.npy')
print(alpha_alpha_h3lin_k7_test)

[[[[ 6.90638315e-310 -5.46672834e-002 -2.79957705e-018]
   [ 1.76776695e-001  0.00000000e+000 -9.29975119e-019]
   [-2.19863239e-001  2.74846234e-002  6.18178402e-019]]

  [[ 0.00000000e+000  1.82683933e-001  2.85820359e-014]
   [ 1.82683933e-001 -4.22276720e-001 -1.35406441e-001]
   [ 2.85837638e-014 -1.35406441e-001  3.27488514e-018]]

  [[ 0.00000000e+000  2.85700700e-014  1.27876105e-001]
   [ 2.85653283e-014  1.48140412e-001 -8.30146862e-014]
   [ 1.27876105e-001 -8.30077203e-014  6.63264043e-009]]]


 [[[ 4.63694668e-310  1.82683933e-001  2.85820359e-014]
   [ 1.82683933e-001  3.16132411e-002 -1.35406441e-001]
   [ 2.85837638e-014 -1.35406441e-001  9.99673401e-019]]

  [[ 7.07106781e-001  5.46672834e-002  9.12495476e-019]
   [-5.22912517e-001  0.00000000e+000  5.96370332e-019]
   [ 2.19863239e-001  9.52804037e-002  6.18902472e-019]]

  [[ 0.00000000e+000 -1.35406430e-001 -8.31308980e-014]
   [-1.35406430e-001 -4.22276720e-001  1.20769815e-001]
   [-8.31291768e-014  1.20769815e-00

In [8]:
#h1_npy_h3lin = (alpha_h_h3lin, beta_h_h3lin)
#g2_npy_h3lin = (alpha_alpha_g_h3lin, alpha_beta_g_h3lin, beta_beta_g_h3lin)
#h1_h3lin_k8 = (alpha_h3lin_k8, beta_h3lin_k8)
#g2_h3lin_k8 = (alpha_alpha_h3lin_k8, alpha_beta_h3lin_k8, beta_beta_h3lin_k8)
#h1_h3lin_k9 = (alpha_h3lin_k9, beta_h3lin_k9)
#g2_h3lin_k9 = (alpha_alpha_h3lin_k9, alpha_beta_h3lin_k9, beta_beta_h3lin_k9)
#h1_h3lin_k7test = (alpha_h3lin_k7_test, beta_h3lin_k7_test)
#g2_h3lin_k7test = (alpha_alpha_h3lin_k7_test, alpha_beta_h3lin_k7_test, beta_beta_h3lin_k7_test)

H3_lin = gto.M(atom=[["H", 0, 0, 0],
                     ["H", 0.93, 0, 0], 
                     ["H", -0.93, 0, 0]], basis = 'sto-3g', charge = 0, spin = 1)
uhf=scf.UHF(H3_lin).run()
norbs, nelecas = (3, (2, 1))
ref_ucas = mcscf.UCASSCF(uhf, norbs, nelecas).mc1step()[0]

bond_dims = [10] * 10
noises = [1e-4] *  5 + [1e-5] * 5 + [0]
thrds = [1e-8] * 10
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_uhf_integrals(uhf,
    ncore=0, ncas=None, g2e_symm=1)


h3_dmrg_driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
h3_dmrg_driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo = h3_dmrg_driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=0)
ket = h3_dmrg_driver.get_random_mps(tag="GS", bond_dim=10, nroots=1)
energy = h3_dmrg_driver.dmrg(mpo, ket, n_sweeps=15, bond_dims=bond_dims, noises=noises,
   thrds=thrds, cutoff=0, iprint=0)
print('UHF-DMRG energy = %20.15f' % energy)

#dmrg_driver_test = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
#dmrg_driver_test.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

#mpo_test = dmrg_driver_test.get_qc_mpo(h1e=h1_npy_h3lin, g2e=g2_npy_h3lin, ecore=ecore, iprint=0)
#ket_test = dmrg_driver_test.get_random_mps(tag="GS", bond_dim=10, nroots=1)
#energy_test = dmrg_driver_test.dmrg(mpo_test, ket_test, n_sweeps=15, bond_dims=bond_dims, noises=noises,
#   thrds=thrds, cutoff=0, iprint=0)
#print('UHF-MRA-DMRG energy (k7) = %20.15f' % energy_test)

#dmrg_driver_k8 = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
#dmrg_driver_k8.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

#mpo_k8 = dmrg_driver_k8.get_qc_mpo(h1e=h1_h3lin_k8, g2e=g2_h3lin_k8, ecore=ecore, iprint=0)
#ket_k8 = dmrg_driver_k8.get_random_mps(tag="GS", bond_dim=10, nroots=1)
#energy_k8 = dmrg_driver_k8.dmrg(mpo_k8, ket_k8, n_sweeps=15, bond_dims=bond_dims, noises=noises,
#   thrds=thrds, cutoff=0, iprint=0)
#print('UHF-MRA-DMRG energy (k8)= %20.15f' % energy_k8)

#dmrg_driver_k9 = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
#dmrg_driver_k9.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

#mpo_k9 = dmrg_driver_k8.get_qc_mpo(h1e=h1_h3lin_k9, g2e=g2_h3lin_k9, ecore=ecore, iprint=0)
#ket_k9 = dmrg_driver_k9.get_random_mps(tag="GS", bond_dim=10, nroots=1)
#energy_k9 = dmrg_driver_k9.dmrg(mpo_k9, ket_k9, n_sweeps=15, bond_dims=bond_dims, noises=noises,
#   thrds=thrds, cutoff=0, iprint=0)
#print('UHF-MRA-DMRG energy (k9)= %20.15f' % energy_k9)


#print('Delta E-k7: ', energy-energy_test)
#print('Delta E-k8: ', energy-energy_k8)
#print('Delta E-k9: ', energy-energy_k9)

converged SCF energy = -1.54687750275231  <S^2> = 0.79069825  2S+1 = 2.0402924
UCASSCF energy = -1.5710709994598
UCASCI E = -1.5710709994598  E(CI) = -2.99359038365334  S^2 = 0.7500000
UHF-DMRG energy =   -1.571070999459797


In [11]:
onerdm_pyscf = h3_dmrg_driver.get_1pdm(ket)
twordm_pyscf = h3_dmrg_driver.get_2pdm(ket)

alpha_1rdm=onerdm_pyscf[0]
beta_1rdm=onerdm_pyscf[1]
aa_2rdm = twordm_pyscf[0]
ab_2rdm = twordm_pyscf[1]
bb_2rdm = twordm_pyscf[2]


#print(alpha_1rdm)

#np.save('/workspaces/MRA-OrbitalOptimization/reduced_density_matrices/h3lin_sto3g_min/alpha_rdms.npy', alpha_1rdm)
#np.save('/workspaces/MRA-OrbitalOptimization/reduced_density_matrices/h3lin_sto3g_min/beta_rdms.npy', beta_1rdm)
np.save('/workspaces/MRA-OrbitalOptimization/reduced_density_matrices/h3lin_sto3g_min/alpha_alpha_rdms.npy', aa_2rdm)
np.save('/workspaces/MRA-OrbitalOptimization/reduced_density_matrices/h3lin_sto3g_min/alpha_beta_rdms.npy', ab_2rdm)
np.save('/workspaces/MRA-OrbitalOptimization/reduced_density_matrices/h3lin_sto3g_min/beta_beta_rdms.npy', bb_2rdm)

In [12]:
a_rdm = np.load('/workspaces/MRA-OrbitalOptimization/reduced_density_matrices/h3lin_sto3g_min/alpha_rdms.npy')
b_rdm = np.load('/workspaces/MRA-OrbitalOptimization/reduced_density_matrices/h3lin_sto3g_min/beta_rdms.npy')
aa_rdm = np.load('/workspaces/MRA-OrbitalOptimization/reduced_density_matrices/h3lin_sto3g_min/alpha_alpha_rdms.npy')
print(aa_rdm)

[[[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
   [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
   [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]

  [[ 0.00000000e+00 -9.76456017e-01 -3.01667489e-15]
   [ 9.76456017e-01  0.00000000e+00  2.98723173e-02]
   [ 3.01667489e-15 -2.98723173e-02  0.00000000e+00]]

  [[ 0.00000000e+00 -3.01667489e-15 -1.67393380e-02]
   [ 3.01667489e-15  0.00000000e+00  1.11960826e-16]
   [ 1.67393380e-02 -1.11960826e-16  0.00000000e+00]]]


 [[[ 0.00000000e+00  9.76456017e-01  3.01667489e-15]
   [-9.76456017e-01  0.00000000e+00 -2.98723173e-02]
   [-3.01667489e-15  2.98723173e-02  0.00000000e+00]]

  [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
   [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
   [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]

  [[ 0.00000000e+00  2.98723173e-02  1.11960826e-16]
   [-2.98723173e-02  0.00000000e+00 -6.80464473e-03]
   [-1.11960826e-16  6.80464473e-03  0.00000000e+00]]]


 [[[ 0.00000000e+00  3.0166748

In [16]:
onerdm_pyscf = dmrg_driver.get_1pdm(ket)
twordm_pyscf = dmrg_driver.get_2pdm(ket)

alpha_1rdm=onerdm_pyscf[0]
beta_1rdm=onerdm_pyscf[1]
alpha_alpha_2rdm = twordm_pyscf[0].transpose(0, 3, 1, 2)
alpha_alpha_2rdm = np.swapaxes(alpha_alpha_2rdm, 1, 2)
alpha_beta_2rdm = twordm_pyscf[1].transpose(0, 3, 1, 2)
alpha_beta_2rdm = np.swapaxes(alpha_beta_2rdm, 1, 2)
beta_beta_2rdm = twordm_pyscf[2].transpose(0, 3, 1, 2)
beta_beta_2rdm = np.swapaxes(beta_beta_2rdm, 1, 2)


energy_from_int = np.einsum('ij,ij->', alpha_1rdm, h1e[0]) + np.einsum('ij,ij->', beta_1rdm, h1e[1]) + 0.5 * (np.einsum('ijkl,ijkl->', alpha_alpha_2rdm, dmrg_driver.unpack_g2e(g2e[0])) + 
                                                                                         np.einsum('ijkl,ijkl->', alpha_beta_2rdm, dmrg_driver.unpack_g2e(g2e[1])) + 
                                                                                         np.einsum('ijkl,ijkl->', beta_beta_2rdm, dmrg_driver.unpack_g2e(g2e[2]))) + ecore

print(energy_from_int)


-2.7904000814668306


In [21]:
Diff1 = -1.61432596e+00 - -1.61432484
Diff2 = -1.61432596e+00 - -1.61432182

print(Diff1, Diff2)

-1.1199999998545707e-06 -4.1399999999303105e-06
